In [1]:
import pandas as pd

In [2]:
events = pd.read_csv('data/events.csv')
event_results = pd.read_csv('data/event_results.csv')
event_details = pd.read_csv('data/event_details.csv')

In [3]:
# 1. events table
events['host_acp_code'] = events['club'].str.replace('^.* / ', '', regex=True)
events_norm = events[['eid', 'host_acp_code', 'date', 'rtid', 'type']]

In [4]:
# 2. event_details table
event_details_norm = event_details[['eid', 'finishers', 'dnf']]
# drop 0 participation events -- with no finishers and no dnf
event_details_norm = event_details_norm[
    (event_details_norm.finishers > 0 ) |
    (event_details_norm.dnf > 0  )
]

# some events have dnf = -1, we'll take these as 0
event_details_norm.loc[event_details_norm.dnf < 0, 'dnf'] = 0
event_details_norm['starters'] = event_details_norm.finishers + event_details_norm.dnf

In [28]:
# 3. riders, with most recent club affiliation from results
riders = \
    event_results.merge(events_norm, on='eid') \
    .sort_values('date') \
    .groupby('rusa') \
    .tail(1)

riders['acp_code'] = riders['club / acp code'].str.replace('^.* / ', '', regex=True)
riders['last_name'] = riders['name'].str.replace(',.*$', '', regex=True)
riders['first_name'] = riders['name'].str.replace('^.*, ', '', regex=True)

riders_norm = riders[['rusa', 'acp_code', 'name', 'first_name', 'last_name']]

In [60]:
# 4. event results
event_results_norm = event_results[['cert','rusa','eid','hours','minutes','medal']] \
    .assign(finish_hours = lambda x: x.hours + x.minutes/60)
